In [20]:
import json
import os
import sys

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

sys.path.append(os.path.abspath("../src/"))
from pvgis_utils import estimate_potential, get_pvgis_specific_yield
from scoring import calculate_combined_score

## A few manual PVGIS calls

In [2]:
# Leuven coordinates
lat = 50.879
lon = 4.701

In [3]:
url = "https://re.jrc.ec.europa.eu/api/v5_3/PVcalc"
params = {
    "lat": lat,
    "lon": lon,
    "peakpower": 1,
    "loss": 14,    
    "angle": 30,          
    "aspect": 0,    #south orientation       
    "outputformat": "json"
}

In [4]:
response = requests.get(url, params=params)
response.raise_for_status()

data = response.json()

print(json.dumps(data["outputs"]["totals"], indent=2))

{
  "fixed": {
    "E_d": 2.92,
    "E_m": 88.9,
    "E_y": 1066.77,
    "H(i)_d": 3.64,
    "H(i)_m": 110.83,
    "H(i)_y": 1329.97,
    "SD_m": 4.21,
    "SD_y": 50.54,
    "l_aoi": -3.13,
    "l_spec": "1.72",
    "l_tg": -5.35,
    "l_total": -19.79
  }
}


In [5]:
# test with different angles
for angle in [0, 10, 20, 30, 40]:
    params["angle"] = angle
    r = requests.get(url, params=params)
    r.raise_for_status()
    d = r.json()
    Ey = d["outputs"]["totals"]["fixed"]["E_y"]
    print(f"tilt={angle}° → {Ey:.1f} kWh/kWp/year")

tilt=0° → 897.2 kWh/kWp/year
tilt=10° → 975.8 kWh/kWp/year
tilt=20° → 1032.6 kWh/kWp/year
tilt=30° → 1066.8 kWh/kWp/year
tilt=40° → 1077.6 kWh/kWp/year


In [6]:
# test with different orientations
params["angle"] = 30 
for aspect in [0, -90, 90, 180]: #[south, east, west, north]
    params["aspect"] = aspect
    r = requests.get(url, params=params)
    r.raise_for_status()
    d = r.json()
    Ey = d["outputs"]["totals"]["fixed"]["E_y"]
    print(f"orientation={aspect}° → {Ey:.1f} kWh/kWp/year")

orientation=0° → 1066.8 kWh/kWp/year
orientation=-90° → 868.1 kWh/kWp/year
orientation=90° → 853.6 kWh/kWp/year
orientation=180° → 608.2 kWh/kWp/year


## Test with function estimate_potential

In [ ]:
# The city hall of Leuven
lat_stadhuis = 50.880
lon_stadhuis = 4.716
area_m2_stadhuis = 1_726

result_stadhuis = estimate_potential(lat_stadhuis, lon_stadhuis, area_m2_stadhuis)
result_stadhuis

{'roof_area_m2': 1726.0,
 'usable_panel_area_m2': 1035.6,
 'kwp': 196.76399999999998,
 'specific_yield_kwh_per_kwp': 1065.92,
 'kwh_year': 209734.68288,
 'co2_tons': 48.238977062400004}

In [18]:
# 30cc/schouwburg
lat_schouwburg = 50.880
lon_schouwburg = 4.705
area_m2_schouwburg = 1_607

result_schouwburg = estimate_potential(lat_schouwburg, lon_schouwburg, area_m2_schouwburg)
result_schouwburg

{'roof_area_m2': 1607.0,
 'usable_panel_area_m2': 964.1999999999999,
 'kwp': 183.198,
 'specific_yield_kwh_per_kwp': 1066.77,
 'kwh_year': 195430.13046000001,
 'co2_tons': 44.94893000580001}

## Test on top 10 sample roofs - plug in when ``candidates`` df is available

In [ ]:
top10_sample = candidates.nlargest(10, "area_m2").copy()

top10_sample_wgs = top10_sample.to_crs(4326).copy()

results = []
for idx, row in top10_sample_wgs.iterrows():
    geom = row.geometry
    cen = geom.centroid
    lat = cen.y
    lon = cen.x
    area = row["area_m2"]

    estimated_potential = estimate_potential(lat, lon, area)
    estimated_potential["idx"] = idx
    results.append(estimated_potential)

result_df = pd.DataFrame(results).set_index("idx")
sample_with_pv_results = top10_sample.join(result_df)

sample_with_pv_results[[
    "area_m2",
    "usable_panel_area_m2",
    "kwp",
    "specific_yield_kwh_per_kwp",
    "kwh_year",
    "co2_tons"
]]

In [ ]:
df = sample_with_pv_results.copy()

df_with_scores = calculate_combined_score(
    df,
    area_col="area_m2",
    yield_col="specific_yield_kwh_per_kwp",
    weight_area=0.4,
    weight_yield=0.4,
    weight_orient=0.2
)

## Visualization

In [ ]:
df_plot = df_with_scores.sort_values("co2_tons", ascending=False).copy()

labels = [f"Roof {i}" for i in range(1, len(df_plot) + 1)]

plt.figure(figsize=(10, 5))
plt.bar(labels, df_plot["co2_tons"])
plt.ylabel("CO₂ savings (tons/year)")
plt.xlabel("Top 10 sample roofs")
plt.title("Estimated annual CO2 savings for 10 large roofs in Leuven")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()